In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
import json
import sys
from pathlib import Path

import ipywidgets as w
import ipyvuetify as v
import pdblp
import numpy as np
import pandas as pd
from ipydatagrid import DataGrid
from sklearn.linear_model import LinearRegression

if (path := 'C:\\dev\\projects\\credit\\backend') not in sys.path:
    sys.path.append(path)
    
if (path := 'D:\\dev\\projects\\credit\\backend') not in sys.path:
    sys.path.append(path)

# from api.blp import BlpQuery
import utils

In [26]:
root_path = Path('/dev/projects/credit/backend/')
data_path = root_path / 'data'

In [27]:
### Bloomberg Connection

In [28]:
with open(data_path / 'pairs.json') as f:
    pairs = json.load(f)

In [29]:
indices = pd.read_csv(data_path / 'indices.csv')
indices_history = pd.read_csv(data_path / 'indices_history.csv', parse_dates=['date'], dayfirst=True)

In [30]:
indices = indices.assign(
    combined_name=indices.drop('ticker', axis=1).apply(lambda x: '_'.join(x.dropna()), axis=1),
)

In [31]:
# dg = DataGrid(
#     dataframe=indices, 
#     selection_mode='row',
#     base_column_size=90,
#     column_widths={
#         'ticker': 120,
#         'combined_name': 300
#     }
# )
# box = w.VBox(
#     children=[
#         dg,
#         out
#     ]
# )


# def get_tickers():
#     c = indices.columns.get_loc('ticker')
#     return [indices.iloc[dict_['r'], c] for dict_ in dg.selected_cells if dict_['c'] == c]


# def fn(change):
#     if len(dg.selections) == 2:
#         fig = make_fig(get_tickers())
#         with out:
#             out.clear_output()
#             display(fig)
            
            
# dg.observe(fn)

In [32]:
def make_fig(tickers):
    
    df = indices_history.loc[indices_history['security'].isin(tickers)] \
                                                        .pivot(index='date', columns='security') \
                                                        .droplevel(level=0, axis=1).dropna()
    ratio = (df.iloc[:, 0] / df.iloc[:, 1]).rename(' / '.join(tickers))
    fig = ratio.plot(backend='plotly',  height=500, template='plotly_white')
    return fig.update_layout(
        legend=
        dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="left",
            x=0
        )
    )

In [33]:
idx1_sel = v.Autocomplete(
    v_model=None,
    items=indices['combined_name'].to_list(),
    label='idx1',
    dense=True,
    outlined=True
)

idx2_sel = v.Autocomplete(
    v_model=None,
    items=indices['combined_name'].to_list(),
    label='idx2',
    dense=True,
    outlined=True
)

chart_btn1 = v.Btn(
    left=True,
    children=[
        v.Icon(children=['mdi-chart-line']),
        'Chart'
    ],
)
out1 = w.Output()

In [34]:
param_box1 = v.Container(
    children=[
        v.Row(
            children=[
                v.Col(
                    cols=4,
                    children=[idx1_sel]
                ),
                v.Col(
                    cols=4,
                    children=[idx2_sel]
                ),
                v.Col(
                    cols=2,
                    children=[chart_btn1]
                ),
            ]
        )
    ]    
)

In [35]:
tab1 = w.VBox(
    children=[
        param_box1,
        out1,
    ]
)

In [36]:
def on_click_chart1(widget, event, data):
    widget.loading = True
    try:
        names = [idx1_sel.v_model, idx2_sel.v_model]
        tickers = indices.loc[indices['combined_name'].isin(names), 'ticker'].to_list()
        fig = make_fig(tickers)
        with out1:
            out1.clear_output()
            display(fig)
    except Exception:
        pass
    finally:
        widget.loading = False

chart_btn1.on_event('click', on_click_chart1)

In [37]:
### Indices Ratio

In [38]:
area_sel = v.Autocomplete(
    v_model=None,
    items=[*pairs],
    label='area',
    dense=True,
    outlined=True
)

pair_sel = v.Autocomplete(
    v_model=None,
    items=None,
    label='pair',
    dense=True,
    outlined=True
)

chart_btn2 = v.Btn(
    left=True,
    children=[
        v.Icon(children=['mdi-chart-line']),
        'Chart'
    ],
)

out2 = w.Output()

In [39]:
def on_change_populate_dd(widget, event, data):
    pair_sel.items = [*pairs[widget.v_model]]

    
area_sel.on_event('change', on_change_populate_dd)

In [40]:
param_box2 = v.Container(
    children=[
        v.Row(
            children=[
                v.Col(
                    cols=2,
                    children=[area_sel]
                ),
                v.Col(
                    cols=2,
                    children=[pair_sel]
                ),
                v.Col(
                    cols=2,
                    children=[chart_btn2]
                ),
            ]
        )
    ]    
)

In [41]:
tab2 = w.VBox(
    children=[
        param_box2,
        out2,
    ]
)

In [42]:
def on_click_chart2(widget, event, data):
    widget.loading = True
    try:
        tickers = pairs[area_sel.v_model][pair_sel.v_model]
        fig = make_fig(tickers)
        with out2:
            out2.clear_output()
            display(fig)
    except Exception as e:
        print(e)
    finally:
        widget.loading = False

chart_btn2.on_event('click', on_click_chart2)

In [43]:
class Tab:
    def __init__(self, tab, name=None):
        self.tab = tab
        self.name = name
        
    def _ipython_display_(self):
        display(self.tab)

In [44]:
class MultiTab:
    def __init__(self, tabs):
        self.tabs = tabs
        self.build()
    
    def build(self):
        self.tabs_bar = v.Tabs(
            # dark=True,
            v_model=None,
            children=[v.Tab(children=[c.name]) for c in self.tabs]
        )

        self.tabs_items = v.TabsItems(
            v_model=None,
            # dark=True,
            children=[v.TabItem(children=[t.tab]) for t in self.tabs]
        )
        
        self.box = v.Card(
            children=[
                self.tabs_bar,
                self.tabs_items
            ]
        )
        
        w.jslink((self.tabs_bar, 'v_model'), (self.tabs_items, 'v_model'))
        
    def _ipython_display_(self):
        display(self.box)

In [45]:
app = MultiTab(
    tabs=[
        Tab(tab2, name='pairs'),
        Tab(tab1, name='details')
    ]
)

### Credit Monitor

In [46]:
app

Card(children=[Tabs(children=[Tab(children=['pairs']), Tab(children=['details'])], v_model=None), TabsItems(ch…